# ECE 661: Homework 4

## Pruning and Fixed-point Quantization

John Coogan

## True/False Questions (15 pts)

**Problem 1.1 (3 pts)** Using sparsity-inducing regularizers like L-1 in DNN optimization with SGD guarantees
exact zero values in weight elements, making further pruning unnecessary.

> False, L-1 regularization incentivizes zero weight values over L-2 regularization but does not guarantee that weights will be zero.

**Problem 1.2 (3 pts)** While weight pruning and weight quantization both compress DNN models, they
interfere with each other’s processes.

> False, pruning and quantization do not interfere with each other. Pruning is the process of ignoring or dropping weights at certain points during training while quantization is the more efficient representation of these weights. These processes can work together. 

**Problem 1.3 (3 pts)** In weight pruning techniques, the distribution of the remaining weights affects the
inference latency.

>  True, certain pruning methods can lead to sparse distributions of weights which can be challenging for hardware that may not support or be optimized for sparse matrix operations. That said, if you achieve structured sparsity, standard CPU/GPU hardware may be able to capitalize on this optimization for better inference latency.

**Problem 1.4 (3 pts)** Group Lasso can lead to structured sparsity on DNNs, which is more hardwarefriendly.
The idea of Group Lasso comes from applying L-2 regularization to the L-1 norm of all of the
groups.

> False, this statement is almost entirely correct but the idea of Group Lasso comes from applying L-1 regularization to the L-2 norm of all the groups. This results in strucutred sparsity by inducing all-zero groups.

**Problem 1.5 (3 pts)** Using soft thresholding operator will lead to better results comparing to using L-1
regularization directly as it solves the "bias" problem of L-1.

> True, soft thresholding partially solves this problem. The bias problem is that the absolute value of all large weights is reduced by Lambda. Soft thresholding only applies this proximal smothing to small weights and leaves large weights untouched, this allows for the preservation of variance.

## Lab 1: Sparse Optimization of Linear Models (30 pts)

By now you have seen multiple ways to induce a sparse solution in the optimization process. This problem
will provide you some examples under linear regression setting so that you can compare the effectiveness
of different methods. For this problem, consider the case where we are trying to find a sparse weight W
that can minimize L =
P
i(XiW − yi)2. Specifically, we have Xi ∈ R1×5, W ∈ R5×1 and ||W||0 ≤ 2.
For Problem (a) - (f), consider the case where we have 3 data points: (X1 = [−1, 2, 1, 1,−1], y1 = 5);
(X2 = [−2, 1,−2, 0, 2], y2 = 1); (X3 = [1, 0,−2,−2,−1], y3 = 1). For stability the objective L should be
minimized through full-batch gradient descent, with initial weight W0 set to [0; 0; 0; 0; 0] and use learning
rate μ = 0.02 throughout the process. Please run gradient descent for 200 steps for all the following
problems. For log(L) plot, please use matplotlib.pyplot.yscale(’log’)

**(a) (4 pts)** Theoretical analysis: with learning rate μ, suppose the weight you have after step k isWk,
derive the symbolic formulation ofweightWk+1 after step k+1 of full-batch gradient descent with
Xi, yi, i ∈ {1, 2, 3}. (Hint: note the loss L we have is defined differently from standard MSE loss.)

The standard update equation holds here:

$W^{(k+1)} = W^{(k)} - \mu \nabla L(W) : L(W) = \sum_{i} (X_{i}W-y_{i})^{2}$

$\nabla L(W) = \frac{d}{dW}[\sum_{i} (X_{i}W-y_{i})^{2}] = 2\sum_{i}X_{i}(X_{i}W-y_{i})$

Therefore, for full batch from 1 to 3:

$W^{(k+1)} = W^{(k)} - 2\mu\sum_{i=1}^{3}X_{i}(X_{i}W-y_{i})$


**(b) (3 pts)** In Python, directly minimize the objective L without any sparsity-inducing regularization/
constraint. Plot the value of log(L) vs. #steps throughout the training, and use another
figure to plot how the value of each element in W is changing throughout the training. From
your result, is W converging to an optimal solution? Is W converging to a sparse solution?

#### Code for Gradient Descent and Training:

![alt text](<Report Images/grad_desc_func.png>)


![alt text](<Report Images/train_func.png>)

![alt text](<Report Images/vanila_grad_desc.png>)

The W values are converging at around 25 epochs, this solution appears to be an optimal solution because the loss is low but it is not a spase solution because none of the W values are zero.

**(c) (6 pts)** Since we have the knowledge that the ground-truth weight should have ||W||0 ≤ 2, we
can apply projected gradient descent to enforce this sparse constraint. Redo the optimization
process in (b), this time prune the elements in W after every gradient descent step to ensure
||Wl||0 ≤ 2. Plot the value of log(L) throughout the training, and use another figure to plot
the value of each element in W in each step. From your result, is W converging to an optimal
solution? Is W converging to a sparse solution?

![alt text](<Report Images/projected grad desc.png>)

> Here we can see that W is converging on a sparse solution (only two of the W values are non-zero). This sparsification is at the expense of significant loss increase compared to the base vanilla gradient descent.

**(d) (5 pts)** In this problem we apply ℓ1 regularization to induce the sparse solution. The minimization
objective therefore changes to L + λ||W||1. Please use full-batch gradient descent to minimize
this objective, with λ = {0.2, 0.5, 1.0, 2.0} respectively. For each case, plot the value of log(L)
throughout the training, and use another figure to plot the value of each element in W in each
step. From your result, comment on the convergence performance under different λ.

> Below are the individual plots for each lambda value for L1 regularization:


![alt text](<Report Images/L1_02.png>)

![alt text](<Report Images/L1_05.png>)

![alt text](<Report Images/L1_1.png>)

![alt text](<Report Images/L1_2.png>)


>We also have all the L1 regularization plots together to see the differences:

![alt text](<Report Images/L1_All.png>)

>What we see with increasing L1 strength is that the model weights are increasingly pushed towards zero leading to a sparser solution but this comes at the cost of higher loss values. Our model becomes more efficient (sparse) but at the expense of accuracy.

> 

**(e) (6 pts)** Here we optimize the same objective as in (d), this time using proximal gradient update.
Recall that the proximal operator of the ℓ1 regularizer is the soft thresholding function. Set the
threshold in the soft thresholding function to {0.004, 0.01, 0.02, 0.04} respectively. Plot the value
of log(L) throughout the training, and use another figure to plot the value of each element in W
in each step. Compare the convergence performance with the results in (d). (Hint: Optimizing
L + λ||W||1 using gradient descent with learning rate μ should correspond to proximal gradient
update with threshold μλ)

> We can take a look at several individual plot for proximal gradient update with varying thresholding functions.

![alt text](<Report Images/prox_02.png>)

![alt text](<Report Images/prox_05.png>)

![alt text](<Report Images/prox_1.png>)

![alt text](<Report Images/prox_2.png>)

> we also have each loss for proximal gradient update plotted together:

![alt text](<Report Images/prox_all.png>)

> We can see that the proximal term succeeds in allowing for smoother convergence toward the overal objective. We see the same tradeoff for L1 regularization in that we can incentivize sparser solutions at the expense of loss.

**(f) (6 pts)** Trimmed ℓ1 (Tℓ1) regularizer is proposed to solve the “bias” problem of ℓ1. For simplicity
you may implement the Tℓ1 regularizer as applying a ℓ1 regularization with strength λ on the 3
elements of W with the smallest absolute value, with no penalty on other elements. Minimize
L+λTℓ1(W) using proximal gradient update with λ = {1.0, 2.0, 5.0, 10.0} (correspond the soft
thresholding threshold {0.02, 0.04, 0.1, 0.2}). Plot the value of log(L) throughout the training,
and use another figure to plot the value of each element in W in each step. Comment on the
convergence comparison of the Trimmed ℓ1 and the ℓ1. Also compare the behavior of the early
steps (e.g. first 20) between the Trimmed ℓ1 and the iterative pruning.

> Here we look at loss and weight curves for multiple lambda values with trimming:

![alt text](<Report Images/trimmed_1.png>)

![alt text](<Report Images/trimmed_2.png>)

![alt text](<Report Images/trimmed_5.png>)

![alt text](<Report Images/trimmed_10.png>)

> And we can look at all the loss curves on top of each other

![alt text](<Report Images/trimmed_all.png>)

> We can see that the iterative pruning yields very similar weight behavior as our L1 trimming. This is most obvious in our trimming with strength 5, which selects the same weights albeit with lower magnitude. 

## Lab 2: Pruning ResNet-20 model (25 pts)

ResNet-20 is a popular convolutional neural network (CNN) architecture for image classification. Compared
to early CNN designs such as VGG-16, ResNet-20 is much more compact. Thus, conducing the model
compression on ResNet-20 is more challenging.
This lab explores the element-wise pruning of ResNet-20 model on CIFAR-10 dataset. We will observe
the difference between single step pruning and iterative pruning, plus exploring different ways of setting
pruning threshold. Everything you need for this lab can be found in HW4.zip.

**(a) (2pts)** In hw4.ipynb, run through the first three code block, report the accuracy of the floatingpoint
pretrained model.

**(b) (6pts)** Complete the implementation of pruning by percentage function in the notebook. Here
we determines the pruning threshold in each DNN layer by the ‘q-th percentile’ value in the
absolute value of layer’s weight element. Use the next block to call your implemented pruning
by percentage. Try pruning percentage q = 0.3, 0.5, 0.7. Report the test accuracy q. (Hint: You
need to reload the full model checkpoint before applying the prune function with a different q ).

**(c) (6pts)** Fill in the finetune_after_prune function for pruned model finetuning. Make sure the
pruned away elements in previous step are kept as 0 throughout the finetuning process. Finetune
the pruned model with q=0.7 for 20 epochs with the provided training pipeline. Report the
best accuracy achieved during finetuning. Finish the code for sparsity evaluation to check if the
finetuned model preserves the sparsity.

**(d) (5pts)** Implement iterative pruning. Instead of applying single step pruning before finetuning, try
iteratively increase the sparsity of the model before each epoch of finetuning. Linearly increase
the pruning percentage for 10 epochs until reaching 70% in the final epoch (prune (7 × e)%
before epoch e) then continue finetune for 10 epochs. Pruned weight can be recovered during
the iterative pruning process before the final pruning step. Compare performance with (c)

**(e) (6pts)** Perform magnitude-based global iterative pruning. Previously we set the pruning threshold
of each layer following the weight distribution of the layer and prune all layers to the same
sparsity. This will constrain the flexibility in the final sparsity pattern across layers. In this question,
Fill in the global_prune_by_percentage function to perform a global ranking of the weight
magnitude from all the layers, and determine a single pruning threshold by percentage for all the
layers. Repeat iterative pruning to 70% sparsity, and report final accuracy and the percentage of
zeros in each layer.

## Lab 3: Fixed-point quantization and finetuning (30pts)

Besides pruning, fixed-point quantization is another important technique applied for deep neural network
compression. In this Lab, you will convert the ResNet-20 model we used in previous lab into a quantized
model, evaluate is performance and apply finetuning on the model.

**(a) (15 pts)** As is mentioned in lecture 15, to train a quantized model we need to use floatingpoint
weight as trainable variable while use a straight-through estimator (STE) in forward and
backward pass to convert the weight into quantized value. Intuitively, the forward pass of STE
converts a float weight into fixed-point, while the backward pass passes the gradient straightly
through the quantizer to the float weight.

To start with, implement the STE forward function in FP_layers.py, so that it serves as a linear
quantizer with dynamic scaling, as introduced on page 9 of lecture 15. Please follow the comments
in the code to figure out the expected functionality of each line. Take a screen shot of
the finished STE class and paste it into the report. Submission of the FP_layers.py file is not
required. (Hint: Please consider zeros in the weight as being pruned away, and build a mask to
ensure that STE is only applied on non-zero weight elements for quantization. )

**(b) (5 pts)** In hw4.ipynb, load pretrained ResNet-20 model, report the accuracy of the floating-point
pretrained model. Then set Nbits in the first line of block 4 to 6, 5, 4, 3, and 2 respectively, run
it and report the test accuracy you got. (Hint: In this block the line defining the ResNet model
(second line) will set the residual blocks in all three stages to Nbits fixed-point, while keeping
the first conv and final FC layer still as floating point.)

**(c) (5 pts)** With Nbits set to 4, 3, and 2 respectively, run code block 4 and 5 to finetune the quantized
model for 20 epochs. You do not need to change other parameter in the finetune function. For
each precision, report the highest testing accuracy you get during finetuning. Comment on the
relationship between precision and accuracy, and on the effectiveness of finetuning.

**(d) (5 pts)** In practice, we want to apply both pruning and quantization on the DNN model. Here we
explore how pruning will affect quantization performance. Please load the checkpoint of the 70%
sparsity model with the best accuracy from Lab 2, repeat the process in (c), report the accuracy
before and after finetuning, and discuss your observations comparing to (c)’s results.